In [40]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os

In [47]:
import fcn_network as net

In [42]:
path = os.getcwd()
dataname = os.path.join(path, "training_data_test.pt")
data = torch.load(dataname)
inputs = data['windows'].unsqueeze(1)
targets = data['c1']
targets = targets.view(-1, 1, 1, 1)
dataset = torch.utils.data.TensorDataset(inputs, targets)

data_size = len(dataset)
train_size = int(0.7 * data_size)
val_size = int(0.15 * data_size)
test_size = data_size - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

/tmp/ipykernel_456322/2484480134.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(dataname)


In [48]:
model = net.fcn_neural_func()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 50

device = torch.device("cpu")
model.to(device)



fcn_neural_func(
  (fc1): Linear(in_features=213, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=1, bias=True)
)

In [44]:
train_loss = []
validation_loss = []
for epoch in range(num_epochs):
    running_loss = 0.0

    model.train()
    ### Training loop
    for inputs,targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        ### Gradient clipping
        optimizer.step()
        running_loss += loss.item()
        print(running_loss)
    running_loss /= len(train_loader)
    train_loss.append(running_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}")
    ### Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    val_accuracy = 100 * correct / total
    val_loss /= len(val_loader)
    validation_loss.append(val_loss)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

### Test loop
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
test_accuracy = 100 * correct / total
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x45369 and 213x512)